In [6]:
###Import standard packages###
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

###Import functions for binning data for preprocessing###
from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import decoder functions
from Neural_Decoding.decoders import KalmanFilterDecoder

In [7]:
###Load Data###
folder='' #ENTER THE FOLDER THAT YOUR DATA IS IN
# folder='/Users/jig289/Dropbox/MATLAB/Projects/In_Progress/BMI/Processed_Data/' 
data=io.loadmat(folder+'jeb7_decoding.mat')
# spike_times=data['spike_times'] #Load spike times of all neurons
# vels=data['vels'] #Load x and y velocities
# vel_times=data['vel_times'] #Load times at which velocities were recorded

In [148]:
latents_avg = data['latents_avg']
latents = data['latents']
jawPos = data['jawPos']
jawVel = data['jawVel']
jawAcc = data['jawAcc']

In [177]:
X_kf = latents[:,[0,2,3,4,5,6]]
X_kf.shape

(118000, 6)

In [188]:
lag=-1 #What time bin of spikes should be used relative to the output
#(lag=-1 means use the spikes 1 bin before the output)

X_kf = latents[:,[0,1,3,4,5,6]]
y_kf = np.concatenate([jawPos,jawVel,jawAcc],axis=1)
 
num_examples=X_kf.shape[0]

#Re-align data to take lag into account
if lag<0:
    y_kf=y_kf[-lag:,:]
    X_kf=X_kf[0:num_examples+lag,:]
if lag>0:
    y_kf=y_kf[0:num_examples-lag,:]
    X_kf=X_kf[lag:num_examples,:]
    
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.7]
testing_range=[0.7, 0.85]
valid_range=[0.85,1]


#Number of examples after taking into account bins removed for lag alignment
num_examples_kf=X_kf.shape[0]
        
#Note that each range has a buffer of 1 bin at the beginning and end
#This makes it so that the different sets don't include overlapping data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples_kf))+1,np.int(np.round(training_range[1]*num_examples_kf))-1)
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples_kf))+1,np.int(np.round(testing_range[1]*num_examples_kf))-1)
valid_set=np.arange(np.int(np.round(valid_range[0]*num_examples_kf))+1,np.int(np.round(valid_range[1]*num_examples_kf))-1)

#Get training data
X_kf_train=X_kf[training_set,:]
y_kf_train=y_kf[training_set,:]

#Get testing data
X_kf_test=X_kf[testing_set,:]
y_kf_test=y_kf[testing_set,:]

#Get validation data
X_kf_valid=X_kf[valid_set,:]
y_kf_valid=y_kf[valid_set,:]


/opt/anaconda3/envs/main/lib/python3.7/site-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/opt/anaconda3/envs/main/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for addi

In [189]:
print(y_kf.shape)
print(X_kf.shape)

(117999, 6)
(117999, 6)


In [190]:
#Z-score inputs 
X_kf_train_mean=np.nanmean(X_kf_train,axis=0)
X_kf_train_std=np.nanstd(X_kf_train,axis=0)
X_kf_train=(X_kf_train-X_kf_train_mean)/X_kf_train_std
X_kf_test=(X_kf_test-X_kf_train_mean)/X_kf_train_std
X_kf_valid=(X_kf_valid-X_kf_train_mean)/X_kf_train_std

#Zero-center outputs
y_kf_train_mean=np.mean(y_kf_train,axis=0)
y_kf_train=y_kf_train-y_kf_train_mean
y_kf_test=y_kf_test-y_kf_train_mean
y_kf_valid=y_kf_valid-y_kf_train_mean


In [191]:


#Declare model
model_kf=KalmanFilterDecoder(C=1) #There is one optional parameter that is set to the default in this example (see ReadMe)

#Fit model
model_kf.fit(X_kf_train,y_kf_train)

#Get predictions
y_valid_predicted_kf=model_kf.predict(X_kf_valid,y_kf_valid)

#Get metrics of fit (see read me for more details on the differences between metrics)
#First I'll get the R^2
R2_kf=get_R2(y_kf_valid,y_valid_predicted_kf)
print('R2:',R2_kf) #I'm just printing the R^2's of the 3rd and 4th entries that correspond to the velocities
#Next I'll get the rho^2 (the pearson correlation squared)
rho_kf=get_rho(y_kf_valid,y_valid_predicted_kf)
print('rho2:',rho_kf**2) #I'm just printing the rho^2's of the 3rd and 4th entries that correspond to the velocities



R2: [-1.12302844 -1.14962059 -0.03890365 -0.03054121 -1.02966062 -0.30136844]
rho2: [0.10376295 0.00980597 0.00478324 0.00098779 0.00049051 0.00027644]


In [197]:
nIdx = y_kf_valid.shape[0]
nIdx = nIdx - 1001
low = np.random.randint(0, high=nIdx)
randTime = np.arange(low,low+1000)

fig, axs = plt.subplots(3, 2,figsize=(20,14))
feat = 0
axs[0,0].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[0,0].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[0,1].set_title('X Position')

feat = 1
axs[0,1].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[0,1].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[0,1].set_title('Y Position')

feat = 2
axs[1,0].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[1,0].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[1,0].set_title('X Velocity')

feat = 3
axs[1,1].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[1,1].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[1,1].set_title('Y Velocity')

feat = 4
axs[2,0].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[2,0].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[2,0].set_title('X Acceleration')

feat = 5
axs[2,1].plot(y_kf_valid[randTime,feat]+y_kf_train_mean[feat],'b')
axs[2,1].plot(y_valid_predicted_kf[randTime,feat]+y_kf_train_mean[feat],'r')
axs[2,1].set_title('Y Acceleration')

plt.show()